In [ ]:
import pandas as pd
import numpy as np

monthly_prices = pd.read_csv("../data/processed/prices_monthly.csv", index_col=0, parse_dates=True)
monthly_returns = pd.read_csv("../data/processed/returns_monthly.csv", index_col=0, parse_dates=True)
ff5 = pd.read_csv("../data/processed/ff5_monthly_clean.csv", index_col=0, parse_dates=True)
panel = pd.read_csv("../data/processed/panel_monthly_with_ff5.csv", index_col=[0,1], parse_dates=[0])

ff5.index = ff5.index + pd.offsets.MonthEnd(0)

# Momentum
momentum_12_1 = monthly_prices.shift(1) / monthly_prices.shift(13) - 1
momentum_12_1_6_1  = monthly_prices.shift(1) / monthly_prices.shift(7)  - 1
momentum_12_1_3_1  = monthly_prices.shift(1) / monthly_prices.shift(4)  - 1

# 3 month volatility of returns
vol_3m = monthly_returns.rolling(3).std()

X = (momentum_12_1.stack().rename("momentum_12_1")
     .to_frame().join(momentum_12_1_6_1.stack().rename("momentum_12_1_6_1"))
     .join(momentum_12_1_3_1.stack().rename("momentum_12_1_3_1"))
     .join(vol_3m.stack().rename("vol_3m")))

print(X)

ValueError: Other Series must have a name